Taylor, R.H., et al., A RESTful API for exchanging materials data in the AFLOWLIB.org consortium. Computational Materials Science, 2014. 93: p. 178-192. https://doi.org/10.1016/j.commatsci.2014.05.014

https://www.makeuseof.com/what-is-rest-api/

ReST API: Representational State Transfer Application Programing Interface.

How does a ReST API work?

Accesses data through **URIs** (Uniform Resource Identifiers), a string of characters that identify a specific resources.

URL (Uniform resource locator)


In [ ]:
# rest api
import requests
URL = 'https://aflowlib.duke.edu/AFLOWDATA/LIB3_WEB/AgCoMn_pv/T0002.A2BC/'
response = requests.get(URL).json()

In [ ]:
# manual
import json 
from urllib.request import urlopen 
SERVER='http://aflowlib.duke.edu/' # server name
PROJECT='AFLOWDATA/LIB3_RAW/' # project name
#URL=SERVER+’/’+PROJECT # project-layer
#URL=SERVER+’/’+PROJECT+’AlCu_pvMn_pv/’ # set-layer
URL=SERVER+PROJECT+'AlCu_pvMn_pv/T0001.A2BC/' # calculation-layer
entry=json.loads(urlopen(URL+'?format=json').read().decode()) # load
for key in entry: # loop keys
    print("{}={}".format (key, entry[key]))

Example 1:

Generating a free-energy zero-temperature convex hull of OsTc

In [ ]:

from urllib.request import urlopen
url0 = 'http://aflowlib.duke.edu/AFLOWDATA/LIB2_RAW/Os_pvTc_pv/'
url1 = 'http://aflowlib.duke.edu/AFLOWDATA/LIB2_RAW/Os_pvTc_pv/?aflowlib_entries'
entry = urlopen(url1)
jfile = entry.read().decode() # download file
j = jfile.split(',') # parse 
j[-1] = 'h3'
sto = []
ent = []
for key in j:
    urls = url0 + key + '/' + '?stoichiometry'
    # print(urls)
    stoich= urlopen(urls).read().decode() # download stoich file
    # print(stoich)
    sto.append(stoich)
    urle = url0 + key + '/' + '?enthalpy_formation_atom'
    enthalpy = urlopen(urle).read().decode() # download formation file
    ent.append(enthalpy)
    # print(enthalpy)

In [ ]:
stoich_data = sto
temps = []
for items in stoich_data:
    items = items.strip()
    temps.append(items)

enthalpy_data = ent
tempe = []
for items in enthalpy_data:
    items = items.strip()
    tempe.append(items)

with open('sto.txt', 'w') as fp:
    for item in temps:
        # write each item on a new line
        fp.write("%s\n" % item)
with open('ent.txt', 'w') as fp:
    for item in tempe:
        # write each item on a new line
        fp.write("%s\n" % item)

import pandas as pd
read_file1 = pd.read_csv ('sto.txt')
read_file1.to_csv ('sto.csv', index=None)
read_file = pd.read_csv ('ent.txt')
read_file.to_csv ('ent.csv', index=None)

path = 'conca.csv'
df = pd.read_csv(path, header=None)

df = df.drop(df[df.iloc[:,1] > 0].index)
import matplotlib.pyplot as plt
sto = df.iloc[:,0]
ent = df.iloc[:,1]
plt.scatter(sto,ent)

In [ ]:
import pandas as pd
df = pd.read_csv('conca.csv')
df = df.drop(df[df.iloc[:,1] > 0.02].index)
df.to_csv('connew.csv',index=None)

Example 2:

Generating a zero temperature phase-diagram of CoNbSi

In [ ]:
aurl3 = 'http://aflowlib.duke.edu/AFLOWDATA/LIB3_RAW/' # ternary systems
aurl2 = 'http://aflowlib.duke.edu/AFLOWDATA/LIB2_RAW/' # binary systems

CoNbSi = aurl3 + 'CoNb_svSi/'
CoNb = aurl2 + 'CoNb_sv/'
NbSi = aurl2 + 'Nb_svSi/'
CoSi = aurl2 + 'CoSi/'

stoich = '/?stoichiometry/'
enthalpy = '?/enthalpy_formation_atom'

comp = [CoNbSi, CoNb, NbSi, CoSi]

In [ ]:
from urllib.request import urlopen
sto = [[] for _ in range(len(comp))]
ent = [[] for _ in range(len(comp))]
count = 0
for url in comp:
    entry = urlopen(url+'?aflowlib_entries')
    jfile = entry.read().decode()
    j = jfile.split(',')
    j[-1] = j[-1].replace('\n', '')

    for key in j:
        urls = url + key + '/' + '?stoichiometry'
        stoich= urlopen(urls).read().decode()
#     # print(stoich)
        sto[count].append(stoich)
        urle = url+ key + '/' + '?enthalpy_formation_atom'
        enthalpy = urlopen(urle).read().decode()
        ent[count].append(enthalpy)
    count += 1


#     # print(enthalpy)

In [ ]:
sto3 = sto
ent3 = ent
temps = [[] for _ in range(len(sto3))]
count = 0
for items in sto3:
    for item in items:
        item = item.strip()
        temps[count].append(item)
    count += 1

tempe = [[] for _ in range(len(ent3))]
count = 0
for items in ent3:
    for item in items:
        item = item.strip()
        tempe[count].append(item)
    count += 1

In [ ]:
import csv
from itertools import zip_longest

export_data = zip_longest(*temps, fillvalue = '')
with open('sto3.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(('0','1','2','3'))
      wr.writerows(export_data)
myfile.close()
export_data = zip_longest(*tempe, fillvalue = '')
with open('ent3.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(('0','1','2','3'))
      wr.writerows(export_data)
myfile.close()

In [ ]:
dfs = pd.read_csv('sto3.csv')
for i in dfs.columns:
    dfs = pd.concat([dfs, dfs[i].str.split(',', expand=True)], axis=1)
dfs = dfs.iloc[:, 4:]
dfs.to_csv('sto3.csv')
# dfe = pd.read_csv('ent3.csv')


How to plot ternary Graph with Origin:

https://www.youtube.com/watch?v=fMWjoM9w9t4&ab_channel=StatgraphicsTechnologies%2CInc


https://www.originlab.com/fileExchange/details.aspx?fid=777